In [1]:
%%bash
mkdir src
mkdir report
mkdir build
rm -rf sample_data/

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import networkx as nx
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# --- Cấu hình ---
# Kiểm tra xem CUDA có sẵn không và đặt thiết bị tương ứng
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Model LLM (Sử dụng Qwen-1.4-7B-Chat, bạn có thể thay đổi)
# Nếu bạn muốn dùng bản 4B, hãy thay bằng 'Qwen/Qwen-1_8B-Chat' hoặc tương tự
# LLM_MODEL_NAME = "Qwen/Qwen1.5-7B-Chat"
LLM_MODEL_NAME = "Qwen/Qwen3-1.7B" # Dùng bản nhỏ hơn để chạy nhanh hơn trên CPU/ít VRAM
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# Đường dẫn lưu trữ
FAISS_INDEX_PATH = "my_faiss_index.index"
GRAPH_PATH = "my_knowledge_graph.gml"
DOC_STORE_PATH = "doc_store.json" # Để lưu trữ text của chunk, map id với text

# --- Khởi tạo Model ---
print(f"Loading LLM model: {LLM_MODEL_NAME}")
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME, trust_remote_code=True)
# Sử dụng device_map="auto" nếu có nhiều GPU hoặc muốn Transformers tự quyết định
# Đối với một GPU hoặc CPU, chỉ định rõ ràng là tốt nhất
if DEVICE == "cuda":
    llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        torch_dtype="auto", # Sử dụng bfloat16 nếu GPU hỗ trợ để tiết kiệm VRAM
        device_map="auto",  # Để transformers tự phân bổ lên GPU
        trust_remote_code=True
    )
else: # CPU
     llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        torch_dtype=torch.float32, # CPU thường dùng float32
        trust_remote_code=True
    ).to(DEVICE)

print(f"Loading embedding model: {EMBEDDING_MODEL_NAME}")
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)

# --- Helper Functions ---
def get_llm_response(prompt_text, max_new_tokens=250):
    messages = [{"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt_text}]
    text = llm_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = llm_tokenizer([text], return_tensors="pt").to(DEVICE)

    generated_ids = llm_model.generate(
        model_inputs.input_ids,
        max_new_tokens=max_new_tokens,
        # pad_token_id=llm_tokenizer.eos_token_id # Quan trọng với một số model
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = llm_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response.strip()

def get_embeddings(texts):
    return embedding_model.encode(texts, convert_to_tensor=True, device=DEVICE)

# --- KAG-Builder ---
class KAGBuilder:
    def __init__(self, chunk_size=500, chunk_overlap=50):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        self.graph = nx.Graph()
        self.doc_store = {} # id_chunk -> text_chunk

    def _extract_entities_simple(self, text_chunk):
        # Đơn giản hóa: trích xuất các từ viết hoa hoặc cụm từ đáng chú ý
        # Trong thực tế, bạn sẽ dùng LLM cho việc này
        prompt = f"""
        Extract up to 3 key entities (people, organizations, locations, specific concepts) from the following text.
        Return them as a comma-separated list. If no clear entities, return "None".

        Text:
        "{text_chunk}"

        Entities:
        """
        entities_str = get_llm_response(prompt, max_new_tokens=50)
        if entities_str.lower() == "none":
            return []
        return [e.strip() for e in entities_str.split(',') if e.strip()]


    def build_from_texts(self, texts_with_sources):
        """
        texts_with_sources: list of tuples, e.g., [("content of doc1", "doc1_id"), ...]
        """
        all_chunks = []
        chunk_id_counter = 0

        for text_content, source_id in texts_with_sources:
            chunks = self.text_splitter.split_text(text_content)
            for i, chunk_text in enumerate(chunks):
                chunk_id = f"{source_id}_chunk_{i}"
                self.doc_store[chunk_id] = chunk_text
                all_chunks.append({"id": chunk_id, "text": chunk_text, "source": source_id})

                # Thêm chunk vào đồ thị
                self.graph.add_node(chunk_id, type="chunk", source=source_id, text=chunk_text[:100]+"...") # Lưu 1 phần text để debug

                # Trích xuất và thêm thực thể (đơn giản)
                entities = self._extract_entities_simple(chunk_text)
                for entity_name in entities:
                    # Chuẩn hóa tên thực thể (ví dụ: viết thường)
                    normalized_entity = entity_name.lower().strip()
                    if not self.graph.has_node(normalized_entity):
                        self.graph.add_node(normalized_entity, type="entity")
                    self.graph.add_edge(chunk_id, normalized_entity, type="mentions")
                
                chunk_id_counter += 1
                if chunk_id_counter % 10 == 0:
                    print(f"Processed {chunk_id_counter} chunks...")


        # Tạo embeddings cho tất cả các chunk
        chunk_texts_for_embedding = [chunk['text'] for chunk in all_chunks]
        chunk_ids_for_embedding = [chunk['id'] for chunk in all_chunks]

        if not chunk_texts_for_embedding:
            print("No chunks to build index from.")
            return

        print("Generating embeddings for chunks...")
        embeddings = get_embeddings(chunk_texts_for_embedding).cpu().numpy()

        # Xây dựng FAISS index
        dimension = embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatL2(dimension)
        self.faiss_index = faiss.IndexIDMap(self.faiss_index) # Để map với ID của chunk

        # Tạo một mảng các ID số cho FAISS
        # Chúng ta sẽ lưu mapping từ id số này về id string của chunk
        self.faiss_id_to_chunk_id = {i: chunk_id for i, chunk_id in enumerate(chunk_ids_for_embedding)}
        numeric_ids = [i for i in range(len(chunk_ids_for_embedding))]
        
        self.faiss_index.add_with_ids(embeddings, numeric_ids)
        print(f"FAISS index built with {self.faiss_index.ntotal} vectors.")

        # Lưu trữ
        faiss.write_index(self.faiss_index, FAISS_INDEX_PATH)
        nx.write_gml(self.graph, GRAPH_PATH)
        import json
        with open(DOC_STORE_PATH, 'w') as f:
            json.dump({"doc_store": self.doc_store, "faiss_id_map": self.faiss_id_to_chunk_id}, f)
        print("Builder process completed and artifacts saved.")


# --- KAG-Solver ---
class KAGSolver:
    def __init__(self, top_k_retrieval=3):
        self.faiss_index = faiss.read_index(FAISS_INDEX_PATH)
        self.graph = nx.read_gml(GRAPH_PATH)
        import json
        with open(DOC_STORE_PATH, 'r') as f:
            saved_data = json.load(f)
            self.doc_store = saved_data['doc_store']
            # faiss_id_map từ string (do JSON) sang int
            self.faiss_id_to_chunk_id = {int(k): v for k, v in saved_data['faiss_id_map'].items()}

        self.top_k = top_k_retrieval
        self.reasoning_prompt_template = PromptTemplate(
            input_variables=["original_query", "history", "current_sub_question", "retrieved_context"],
            template="""
            Bạn là một trợ lý AI giúp trả lời một câu hỏi phức tạp bằng cách chia nhỏ nó ra.
            Câu hỏi gốc: {original_query}

            Lịch sử suy luận (các câu hỏi phụ trước đó và câu trả lời của chúng):
            {history}

            Câu hỏi phụ hiện tại: {current_sub_question}

            Ngữ cảnh được truy xuất cho câu hỏi phụ hiện tại:
            ---
            {retrieved_context}
            ---

            Dựa trên ngữ cảnh được truy xuất và lịch sử suy luận, hãy trả lời Câu hỏi phụ hiện tại.
            Nếu ngữ cảnh không đủ, hãy nêu rõ điều đó và gợi ý những gì có thể còn thiếu.
            Câu trả lời:
            """
        )
        self.synthesis_prompt_template = PromptTemplate(
            input_variables=["original_query", "reasoning_trace"],
            template="""
            Dựa trên câu hỏi gốc sau và chuỗi suy luận từng bước,
            hãy đưa ra một câu trả lời tổng hợp đầy đủ cho câu hỏi gốc.
            Kết hợp thông tin một cách mạch lạc.

            Câu hỏi gốc: {original_query}

            Chuỗi suy luận:
            {reasoning_trace}

            Câu trả lời tổng hợp cuối cùng:
            """
        )

    def _retrieve_from_vector_db(self, query_text):
        query_embedding = get_embeddings([query_text]).cpu().numpy()
        distances, indices = self.faiss_index.search(query_embedding, self.top_k)
        
        retrieved_chunk_texts = []
        for i in range(len(indices[0])):
            faiss_numeric_id = indices[0][i]
            if faiss_numeric_id != -1: # FAISS trả về -1 nếu không đủ k kết quả
                chunk_id_str = self.faiss_id_to_chunk_id.get(faiss_numeric_id)
                if chunk_id_str and chunk_id_str in self.doc_store:
                    retrieved_chunk_texts.append(f"Chunk ID: {chunk_id_str}\nContent: {self.doc_store[chunk_id_str]}\n---")
        return "\n".join(retrieved_chunk_texts)

    def _retrieve_from_kg(self, entities):
        # Đơn giản: tìm các chunk liên quan đến thực thể
        kg_info = []
        for entity in entities:
            normalized_entity = entity.lower().strip()
            if self.graph.has_node(normalized_entity):
                kg_info.append(f"Knowledge Graph information for entity '{entity}':")
                for neighbor in self.graph.neighbors(normalized_entity):
                    if self.graph.nodes[neighbor]['type'] == 'chunk':
                        kg_info.append(f"  - Mentioned in chunk: {neighbor} (Source: {self.graph.nodes[neighbor].get('source', 'N/A')})")
                        # Bạn có thể lấy thêm text của chunk này từ self.doc_store nếu cần
        return "\n".join(kg_info)

    def _plan_steps(self, original_query):
        # Sử dụng LLM để chia câu hỏi thành các bước
        # Cho baseline, chúng ta có thể yêu cầu 3 bước
        prompt = f"""
        Phân tích câu hỏi phức tạp sau thành một chuỗi gồm 3 câu hỏi phụ đơn giản hơn.
        Mỗi câu hỏi phụ nên dựa trên câu trước đó để giúp trả lời câu hỏi gốc.
        Trả về các câu hỏi phụ dưới dạng danh sách được đánh số.

        Câu hỏi gốc: "{original_query}"

        Các câu hỏi phụ:
        1. ...
        2. ...
        3. ...
        """
        plan_str = get_llm_response(prompt, max_new_tokens=150)
        # Phân tích plan_str để lấy các câu hỏi con
        sub_questions = []
        for line in plan_str.split('\n'):
            match = re.match(r"^\d+\.\s*(.+)", line)
            if match:
                sub_questions.append(match.group(1).strip())
        
        # Nếu không phân tích được, trả về một câu hỏi mặc định
        if not sub_questions:
            return [original_query] # fallback
        return sub_questions

    def solve(self, original_query, max_steps=3):
        sub_questions = self._plan_steps(original_query)
        if not sub_questions:
            print("Could not plan steps. Answering directly (basic RAG).")
            context = self._retrieve_from_vector_db(original_query)
            # (Tùy chọn: trích xuất thực thể từ original_query và lấy thông tin KG)
            final_prompt = f"Original Query: {original_query}\nContext:\n{context}\nAnswer:"
            return get_llm_response(final_prompt)

        reasoning_history = []
        reasoning_trace_for_synthesis = ""

        for i, sub_q_text in enumerate(sub_questions):
            if i >= max_steps:
                break
            
            print(f"\n--- Step {i+1}: Sub-question: {sub_q_text} ---")

            # 1. Truy xuất từ Vector DB cho câu hỏi con hiện tại
            retrieved_chunks = self._retrieve_from_vector_db(sub_q_text)
            
            # 2. (Tùy chọn) Trích xuất thực thể từ câu hỏi con và truy xuất KG
            # (Đây là phần bạn có thể làm phức tạp hơn)
            # entity_extraction_prompt = f"Extract key entities from this question: \"{sub_q_text}\". Return as comma-separated list."
            # entities_str = get_llm_response(entity_extraction_prompt, max_new_tokens=30)
            # entities_in_sub_q = [e.strip() for e in entities_str.split(',') if e.strip()]
            # kg_context = self._retrieve_from_kg(entities_in_sub_q)
            # combined_context = f"Vector DB Chunks:\n{retrieved_chunks}\n\nKnowledge Graph Context:\n{kg_context}"
            combined_context = f"Retrieved Chunks:\n{retrieved_chunks}" # Giữ đơn giản

            # 3. Tạo prompt và gọi LLM để trả lời câu hỏi con
            current_history_str = "\n".join([f"  - Q: {item['q']}\n    A: {item['a']}" for item in reasoning_history])
            
            step_prompt_input = {
                "original_query": original_query,
                "history": current_history_str if current_history_str else "No previous steps.",
                "current_sub_question": sub_q_text,
                "retrieved_context": combined_context if combined_context else "No context retrieved."
            }
            step_prompt = self.reasoning_prompt_template.format(**step_prompt_input)
            # print(f"DEBUG: Step Prompt:\n{step_prompt}")
            
            sub_answer = get_llm_response(step_prompt, max_new_tokens=300)
            print(f"Sub-answer: {sub_answer}")

            reasoning_history.append({"q": sub_q_text, "a": sub_answer})
            reasoning_trace_for_synthesis += f"Sub-Question {i+1}: {sub_q_text}\nSub-Answer {i+1}: {sub_answer}\n\n"

        # 4. Tổng hợp câu trả lời cuối cùng
        print("\n--- Synthesizing Final Answer ---")
        synthesis_prompt_input = {
            "original_query": original_query,
            "reasoning_trace": reasoning_trace_for_synthesis
        }
        final_prompt = self.synthesis_prompt_template.format(**synthesis_prompt_input)
        # print(f"DEBUG: Synthesis Prompt:\n{final_prompt}")
        final_answer = get_llm_response(final_prompt, max_new_tokens=500)
        return final_answer

# --- Main Execution ---
if __name__ == "__main__":
    # --- Giai đoạn xây dựng (chỉ chạy một lần hoặc khi dữ liệu thay đổi) ---
    # Kiểm tra xem index đã tồn tại chưa để tránh xây dựng lại không cần thiết
    if not os.path.exists(FAISS_INDEX_PATH) or not os.path.exists(GRAPH_PATH):
        print("Building KAG artifacts...")
        builder = KAGBuilder(chunk_size=300, chunk_overlap=30) # Giảm chunk size để có nhiều chunk hơn
        
        # Ví dụ dữ liệu (thay thế bằng dữ liệu thực của bạn)
        sample_docs = [
            (
                "Điều 49. Xử lý học vụ Sau mỗi học kỳ chính, đơn vị đào tạo thực hiện xử lý học vụ. "
                "Kết quả học tập của học kỳ phụ sẽ được tính vào kết quả học tập của học kỳ chính tiếp theo. "
                "1. Cảnh báo học vụ  Đầu mỗi học kỳ, đơn vị đào tạo cảnh báo đối với những sinh viên có "
                "điểm trung bình chung học kỳ đạt từ 0,80 đến dưới 0,85 đối với học kỳ đầu của khóa học; "
                "đạt từ 1,00 đến dưới 1,10 đối với các học kỳ tiếp theo hoặc đạt từ 1,10 đến dưới 1,20 "
                "đối với 2 học kỳ liên tiếp. 2. Thôi học Sinh viên được thôi học nếu có đơn xin thôi học "
                "và được Thủ trưởng đơn vị đào tạo ra quyết định đồng ý. 3. Buộc thôi học Sau mỗi học kỳ, "
                "sinh viên bị buộc thôi học nếu thuộc một trong các trường hợp sau: a) Có điểm trung bình "
                "chung học kỳ đạt dưới 0,80 đối với học kỳ đầu của khóa học; đạt dưới 1,00 đối với các học kỳ "
                "tiếp theo hoặc đạt dưới 1,10 đối với 2 học kỳ liên tiếp; b) Có điểm trung bình chung tích lũy "
                "đạt dưới 1,20 đối với sinh viên năm thứ nhất; dưới 1,40 đối với sinh viên năm thứ hai; "
                "dưới 1,60 đối với sinh viên năm thứ ba hoặc dưới 1,80 đối với sinh viên các năm tiếp theo "
                "và cuối khóa; c) Vượt quá thời gian tối đa được phép học quy định tại khoản 2, Điều 24 "
                "của Quy chế này; d) Bị kỷ luật lần thứ hai vì lý do thi hộ hoặc nhờ người thi hộ theo "
                "quy định tại mục d, khoản 10, Điều 40 của Quy chế này hoặc bị kỷ luật ở mức xóa tên "
                "khỏi danh sách sinh viên của trường.Chậm nhất 1 tháng sau khi sinh viên có quyết định "
                "buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú",
                "doc_vnu"
            )
        ]
        builder.build_from_texts(sample_docs)
    else:
        print("KAG artifacts already exist. Skipping build phase.")

    # --- Giai đoạn giải quyết truy vấn ---
    solver = KAGSolver(top_k_retrieval=2) # Lấy ít chunk hơn cho mỗi bước
    
    # query1 = "Who are the pioneers of AI and what are their main contributions, especially regarding NLP?"
    # print(f"\nSolving Query 1: {query1}")
    # answer1 = solver.solve(query1)
    # print(f"\nFinal Answer for Query 1:\n{answer1}")

    query2 = "Cho tôi hỏi khi nào sinh viên được thôi học ? Khi nào sinh viên bị buộc thôi học? Sau bao lâu sinh viên có quyết định buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú?, trả lời bằng tiếng việt "
    print(f"\nSolving Query 2: {query2}")
    answer2 = solver.solve(query2, max_steps=3) # Giới hạn số bước cho truy vấn này
    print(f"\nFinal Answer for Query 2:\n{answer2}")

c:\Users\huyho\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Loading LLM model: Qwen/Qwen3-1.7B


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


Loading embedding model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
KAG artifacts already exist. Skipping build phase.

Solving Query 2: Sau bao lâu sinh viên có quyết định buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú?, trả lời bằng tiếng việt 


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Step 1: Sub-question: Sau bao lâu sinh viên có quyết định buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú?, trả lời bằng tiếng việt  ---
Sub-answer: <think>
Okay, let's tackle this question. The user is asking how long after a student decides to withdraw (buộc thôi học) the educational institution must notify the student to return to the place of their permanent residence. The answer needs to be in Vietnamese.

First, I check the retrieved chunks. There's a chunk from doc_vnu_chunk_5 that says "1 tháng sau khi sinh viên có quyết định buộc thôi học, đơn vị đào tạo phải thông báo trả về địa phương nơi sinh viên có hộ khẩu thường trú." So that's a direct answer: 1 month after the student decides to withdraw, the institution must notify them to return to their permanent residence.

Another chunk from doc_vnu_chunk_2 talks about the decision to withdraw and the criteria for it, but it doesn't mention the notification period. So the mai